In [ ]:
# the drive

Use cv.CALIB_USE_INTRINSIC_GUESS if you already have an initial guess for cameraMatrix.
Use cv.CALIB_FIX_ASPECT_RATIO if you know the aspect ratio.
Use cv.CALIB_FIX_K3 to ignore higher-order distortion parameters.
Use cv.CALIB_RATIONAL_MODEL only if absolutely necessary (it's computationally expensive).
                                                          

https://drive.google.com/drive/folders/1flAP1RIpKx5HpPp9bP6uMEHpe9Jh4J9i?usp=sharing

In [1]:
import torch
print(torch.cuda.is_available())


False


In [ ]:
%pip uninstall opencv-python opencv-contrib-python -y
%pip install opencv-contrib-python-headless


Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 MB 8.4 MB/s eta 0:00:00


In [2]:
import cv2
print(cv2.getBuildInformation())  # Look for CUDA support



General configuration for OpenCV 4.7.0 =====================================
  Version control:               4.7.0

  Extra modules:
    Location (extra):            D:/a/opencv-python/opencv-python/opencv_contrib/modules
    Version control (extra):     4.7.0

  Platform:
    Timestamp:                   2022-12-29T19:11:10Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import cv2 as cv
import glob
import pickle
import time

def get_user_input():
    chessboard_rows = int(input("Rows vertices (input number of black and white boxes in a row minus 1): "))
    chessboard_cols = int(input("Columns vertices (input number of black and white boxes in a column minus 1): "))
    square_size = float(input("Size of a black box mm: "))
    return (chessboard_cols, chessboard_rows), square_size

def process_image(image_path, chessboardSize, criteria):
    """ Processes a single image: detects and refines chessboard corners using GPU """
    img = cv.imread(image_path)
    if img is None:
        return None  # Skip if image is invalid

    # Convert image to grayscale using GPU
    gray = cv.UMat(cv.cvtColor(img, cv.COLOR_BGR2GRAY))  # Send to GPU

    # Resize for faster processing
    gray = cv.UMat(cv.resize(gray.get(), (640, 480)))  # Get back to CPU temporarily

    ret, corners = cv.findChessboardCorners(gray.get(), chessboardSize, None)
    if not ret:
        return None  # Skip if no corners found

    corners2 = cv.cornerSubPix(gray.get(), corners, (11, 11), (-1, -1), criteria)
    return corners2

def main():
    start_time = time.time()

    chessboardSize, square_size = get_user_input()
    frameSize = (640, 480)

    # Termination criteria for corner detection refinement
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Generate 3D object points
    objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1, 2)
    objp *= square_size

    objpoints = []
    imgpoints = []

    images = glob.glob('/content/drive/MyDrive/Freshly_Captured/*.png')

    for image_path in images:
        result = process_image(image_path, chessboardSize, criteria)
        if result is not None:
            objpoints.append(objp)
            imgpoints.append(result)

    if not objpoints or not imgpoints:
        print("No valid chessboard images found. Exiting.")
        return

    print(f"Using {len(objpoints)} images for calibration...")

    # Compute an initial camera matrix
    cameraMatrix = cv.initCameraMatrix2D(objpoints, imgpoints, frameSize)

    calib_start = time.time()

    # Perform calibration with RATIONAL MODEL (for best accuracy)
    ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(
        objpoints, imgpoints, frameSize, cameraMatrix, None,
        flags = cv.CALIB_RATIONAL_MODEL
    )

    calib_end = time.time()
    print(f"Camera calibration took {calib_end - calib_start:.2f} seconds")

    # Save calibration results
    with open("/content/drive/MyDrive/calidata/calibration.pkl", "wb") as f:
        pickle.dump((cameraMatrix, dist), f)

    with open("/content/drive/MyDrive/calidata/cameraMatrix.pkl", "wb") as f:
        pickle.dump(cameraMatrix, f)

    with open("/content/drive/MyDrive/calidata/dist.pkl", "wb") as f:
        pickle.dump(dist, f)

    # Compute reprojection error
    mean_error = sum(cv.norm(imgpoints[i], cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)[0], cv.NORM_L2) / len(imgpoints[i])
                     for i in range(len(objpoints))) / len(objpoints)

    print(f"\nTotal reprojection error: {mean_error:.6f}")
    print(f"Total execution time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    main()


Rows vertices (input number of black and white boxes in a row minus 1): 8
Columns vertices (input number of black and white boxes in a column minus 1): 6
Size of a black box mm: 25
Using 342 images for calibration...
Camera calibration took 260.01 seconds

Total reprojection error: 0.045380
Total execution time: 277.97 seconds
